In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds

In [24]:
import io
import itertools
import sklearn.metrics
import datetime
import matplotlib.pyplot as plt

In [3]:
mnist_dataset,mnist_info=tfds.load(name='mnist',with_info=True,as_supervised=True)

In [4]:
mnist_train,mnist_test=mnist_dataset['train'],mnist_dataset['test']

In [5]:
no_of_validation_samples=0.1*mnist_info.splits['train'].num_examples
no_of_validation_samples=tf.cast(no_of_validation_samples,tf.int64)

no_of_test_samples=mnist_info.splits['test'].num_examples
no_of_test_samples=tf.cast(no_of_test_samples,tf.int64)

In [6]:
buffer_size=70000
batch_size=128
max_epoch=20

In [7]:
def scale(image,label):
    image=tf.cast(image,tf.float32)
    image/=255.
    return image,label

In [8]:
scaled_train_and_validation_data=mnist_train.map(scale)
scaled_test_data=mnist_test.map(scale)

In [9]:
suffled_train_and_validation_data=scaled_train_and_validation_data.shuffle(buffer_size)

In [10]:
validation_data=suffled_train_and_validation_data.take(no_of_validation_samples)
train_data=suffled_train_and_validation_data.skip(no_of_validation_samples)

In [11]:
train_data=train_data.batch(batch_size)
validation_data=validation_data.batch(no_of_validation_samples)
test_data=scaled_test_data.batch(no_of_test_samples)

In [25]:
for images,labels in validation_data:
    images_val=images.numpy()
    labels_val=images.numpy()

In [12]:
model=tf.keras.Sequential([
    tf.keras.layers.Conv2D(50,5,activation='relu',input_shape=(28,28,1)),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(50,3,activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10)
])

In [13]:
loss_rn=tf.keras.losses.SparseCategoricalCrossentropy(from_logits='softmax')
model.compile(optimizer='adam',loss=loss_rn,metrics=['accuracy'])

In [14]:
early_stopping=tf.keras.callbacks.EarlyStopping(monitor='val_loss',mode='auto',min_delta=0,verbose=0,patience=2,restore_best_weights=True)

In [18]:
log_dir="logs\\fit\\"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback=tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1)

In [19]:
model.fit(train_data,epochs=max_epoch,callbacks=[tensorboard_callback,early_stopping],validation_data=validation_data,verbose=2)

Epoch 1/20


422/422 - 73s - loss: 0.0144 - accuracy: 0.9953 - val_loss: 0.0192 - val_accuracy: 0.9930
Epoch 2/20
422/422 - 70s - loss: 0.0131 - accuracy: 0.9956 - val_loss: 0.0133 - val_accuracy: 0.9955
Epoch 3/20
422/422 - 69s - loss: 0.0117 - accuracy: 0.9962 - val_loss: 0.0129 - val_accuracy: 0.9958
Epoch 4/20
422/422 - 70s - loss: 0.0099 - accuracy: 0.9969 - val_loss: 0.0068 - val_accuracy: 0.9977
Epoch 5/20
422/422 - 74s - loss: 0.0088 - accuracy: 0.9974 - val_loss: 0.0106 - val_accuracy: 0.9965
Epoch 6/20
422/422 - 75s - loss: 0.0076 - accuracy: 0.9976 - val_loss: 0.0061 - val_accuracy: 0.9977
Epoch 7/20
422/422 - 76s - loss: 0.0079 - accuracy: 0.9975 - val_loss: 0.0064 - val_accuracy: 0.9980
Epoch 8/20
422/422 - 83s - loss: 0.0079 - accuracy: 0.9971 - val_loss: 0.0056 - val_accuracy: 0.9978
Epoch 9/20
422/422 - 89s - loss: 0.0057 - accuracy: 0.9981 - val_loss: 0.0038 - val_accuracy: 0.9990
Epoch 10/20
422/422 - 89s - loss: 0.0057 - accuracy: 0.9982 - val_loss: 0.0044 - val_accuracy: 0.9988


In [26]:
%load_ext tensorboard
%tensorboard --logdir 'logs/fit'

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 3184), started 2:12:25 ago. (Use '!kill 3184' to kill it.)

In [22]:
test_loss,test_accuracy=model.evaluate(test_data)

      1/Unknown - 42s 42s/step - loss: 0.0392 - accuracy: 0.9906